In [1]:
import pandas as pd
import numpy as np

import json
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime

In [2]:

# for date and time
from datetime import datetime 
import holidays

# graphs
import matplotlib.pyplot as plt

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

In [4]:
import torch
import torch.nn as nn

# torch dataloaders
import torch.functional as f
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split

In [76]:
import transformers
# Pre-Trained Models imports
import transformers
from transformers import BertTokenizer , AutoModel # Install Transformers if not installed

from sentence_transformers import SentenceTransformer, util

In [341]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


## Data extraction

In [9]:
root_path = '../FMP/'
transcript_folders = os.listdir(root_path)

interested_companies = [ 'AFL', 'FICO', 'WDC', ]

In [37]:
language_data = {}

for i in transcript_folders:

    if i in interested_companies:

        # Now goes through the inside folder
        folders = os.listdir( os.path.join(root_path, i) )
        language_data[i] = {}

        # Now goes through each file in the folder
        for year_folder in folders :
            # print( os.path.join( os.path.join(root_path, i), year_folder) )
            if os.path.isdir( os.path.join( os.path.join(root_path, i), year_folder) ):

                file_in_folder_path = os.path.join(os.path.join( os.path.join(root_path, i), year_folder)) 
                language_data[i][year_folder] = []

                for j in os.listdir(file_in_folder_path) :
                    full_path = os.path.join(file_in_folder_path,j)
                    print(full_path)
                    json_data = pd.read_json(full_path, typ = 'series')
                    language_data[i][year_folder].append(json_data)

../FMP/AFL\2022\2022_1.json
../FMP/AFL\2022\2022_2.json
../FMP/AFL\2022\2022_3.json
../FMP/AFL\2022\2022_4.json
../FMP/AFL\2023\2023-02-02.json
../FMP/FICO\2022\2022_1.json
../FMP/FICO\2022\2022_2.json
../FMP/FICO\2022\2022_3.json
../FMP/FICO\2022\2022_4.json
../FMP/FICO\2023\2023-01-27.json
../FMP/FICO\2023\2023_1.json
../FMP/WDC\2022\2022_2.json
../FMP/WDC\2022\2022_3.json
../FMP/WDC\2022\2022_4.json
../FMP/WDC\2023\2023_1.json
../FMP/WDC\2023\2023_2.json


In [92]:
avg_len = 0
# for i in 
for i in range( len( language_data['AFL']['2022'][0]['content'].split('\n')) ): 

    avg_len +=  len( language_data['AFL']['2022'][0]['content'].split('\n')[i].split('.') ) 

avg_len

663

In [91]:
len( language_data['AFL']['2022'][0]['content'].split('\n')[i].split('.') )

3

In [ ]:
'kk'.replace()

In [82]:
def get_sentences(input_text):

    sentences = []

    for i in input_text:
        i = i.replace('.','<n>').replace('!','<n>').replace('.','<n>')
        seperated_sentences = i.split('<n>')
    
        sentences.extend( seperated_sentences )

    return sentences

In [94]:
len( get_sentences( language_data['AFL']['2022'][0]['content'].split('\n') ) )

663

In [104]:
def create_dataframes( x, company_name,  ):

    data_file = {}
    keys = list(x.keys())

    for i in keys:

        high_level_data = x[i]
        data_file[i] = {}

        for j,quarters in enumerate(high_level_data):

            text_data = quarters['content'].split('\n')
            data_file[i][j+1] = get_sentences(text_data) 
    
    return data_file

In [105]:
new_language_data = {}

for key in list(language_data.keys()):
    
    new_language_data[key] = create_dataframes(language_data[key], key)

In [133]:
len(new_language_data['WDC']['2022'][2] )

554

## Model creation 

In [215]:
finance_keywords = ['revenue', 'profit', 'EPS (earnings per share)', 'growth', 'acquisitions', 'regulatory changes', 'shifts in consumer behavior', 'strategic initiatives', 'plans for future growth',  'potential risks', 'challenges', 'shareholder value',' dividends', 'share buybacks',  'market conditions',  ]

In [186]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )
Language_model = SentenceTransformer('paraphrase-TinyBERT-L6-v2')

In [216]:
finance_embeddings = Language_model.encode(finance_keywords ,convert_to_tensor= True ).to(device)

In [271]:
def compare_with_keywords(text ,keywords_embeddings = finance_embeddings, top_indexes = 5):

    text_embedding = Language_model.encode(text ,convert_to_tensor= True ).to(device)
    # print(text_embedding.shape, keywords_embeddings.shape)

    # checking similarity
    similarity_check = util.pytorch_cos_sim(text_embedding, keywords_embeddings)

    # Now getting the top indexes
    top_similar = torch.sort( torch.max(similarity_check, dim = 1)[0] , descending = True)[1][:top_indexes]

    # print( torch.sort( torch.max(similarity_check, dim = 1)[0] , descending = True)[1][:top_indexes] ) 
    # print( torch.max( similarity_check, dim = 1 )[0].shape , )
    # print( similarity_check.shape, )

    return top_similar

In [283]:
compare_with_keywords(text = new_language_data['AFL']['2022'][2], top_indexes = 10 )

tensor([ 78, 107, 262, 171,  76, 174, 176,  93,  47, 128], device='cuda:0')

In [294]:
new_language_data['AFL']['2022'][2][128]

' We see no interruption in our core margins and return of capital to shareholders, including dividend pattern and share repurchase'

In [321]:
def get_important_text(symbol, data = new_language_data, keywords_embeddings = finance_embeddings, num_sentences = 5 ):

    contents = data[symbol]

    # creating finance embeddings
    # finance_embeddings = Language_model.encode(finance_keywords ,convert_to_tensor= True ).to(device)
    finance_keyword_embeddings = keywords_embeddings

    # to store the top 15 sentences
    important_sentences = {}

    for key in list(contents.keys()):
        
        quarter_contents = contents[key]
        important_quarter_points = {}

        for quarter_keys in list(quarter_contents.keys()):
            # print(quarter_contents)
            texts = quarter_contents[quarter_keys]

            # print(texts)
            # ll

            indexes = list( compare_with_keywords(texts, finance_keyword_embeddings, top_indexes = num_sentences ).detach().cpu().numpy() )
            model_pred_texts = list( map(texts.__getitem__, indexes) )

            # storing them in the quarter points
            important_quarter_points[quarter_keys] = (model_pred_texts)
        
        important_sentences[key] = important_quarter_points
    
    return important_sentences

In [328]:
AFL_important_texts = get_important_text('AFL')

In [387]:
def give_report_2(full_data, company):

    data = full_data

    print(f" The highlights of the discussion we are going to see for the company '{company}' ")

    for year in list(data.keys()):

        print(f" \n We are currently looking in the year : {year}")
        year_text = data[year]

        # Printing the model indentified important points

        for quarter_keys in list(year_text.keys()):
            print(f" \n Currently seeing important points for the quarter :  ")

            for i,points in enumerate( year_text[quarter_keys] ):
                print(f" point {i+1} :  {points} ")

In [388]:
give_report_2(AFL_important_texts, 'AFL')

 The highlights of the discussion we are going to see for the company 'AFL' 
 
 We are currently looking in the year : 2022
 
 Currently seeing important points for the quarter :  
 point 1 :   At the same time, we know that consumers habits and buying preferences have been evolving 
 point 2 :   But we think right now, there's certain market conditions influencing it 
 point 3 :  2% increase in the first quarter cash dividend 
 point 4 :   I'll also provide some perspective on market and economic conditions in Japan and in the U 
 point 5 :   On that note, as part of the ongoing collaboration and governance framework of the strategic alliance 
 
 Currently seeing important points for the quarter :  
 point 1 :   At the same time, we know consumers' habits and buying preferences have been evolving 
 point 2 :   economic conditions 
 point 3 :   sales question 
 point 4 :   We have 3 important business initiatives underway that are essential to the future growth of the company 
 point 5

In [385]:
def give_report(full_data, company):

    data = full_data

    print(f" The highlights of the discussion we are going to see for the company {color.UNDERLINE}{color.BOLD}{color.YELLOW}'{company}'{color.END} ")

    for year in list(data.keys()):

        print(f" \n We are currently looking in the year : {color.UNDERLINE}{color.BOLD}{color.GREEN}{year}{color.END} ")
        year_text = data[year]

        # Printing the model indentified important points

        for quarter_keys in list(year_text.keys()):
            print(f" \n Currently seeing {color.BOLD}{color.PURPLE}important points{color.END} for the {color.BOLD}{color.CYAN}quarter : {quarter_keys} {color.END} ")

            for i,points in enumerate( year_text[quarter_keys] ):
                print(f" {color.UNDERLINE}{color.BOLD}point {i+1}{color.END} :  {points} ")

In [386]:
give_report(AFL_important_texts, 'AFL')

 The highlights of the discussion we are going to see for the company 'AFL' 
 
 We are currently looking in the year : 2022 
 
 Currently seeing important points for the quarter : 1  
 point 1 :   At the same time, we know that consumers habits and buying preferences have been evolving 
 point 2 :   But we think right now, there's certain market conditions influencing it 
 point 3 :  2% increase in the first quarter cash dividend 
 point 4 :   I'll also provide some perspective on market and economic conditions in Japan and in the U 
 point 5 :   On that note, as part of the ongoing collaboration and governance framework of the strategic alliance 
 
 Currently seeing important points for the quarter : 2  
 point 1 :   At the same time, we know consumers' habits and buying preferences have been evolving 
 point 2 :   economic conditions 
 point 3 :   sales question 
 point 4 :   We have 3 important business initiatives underway that are essential to the future growth of the company 
 po

In [393]:
def full_pipeline(company_symbol, data = new_language_data, num_points = 5):
    name = company_symbol

    company_important_texts = get_important_text(name, num_sentences = num_points)
    give_report(company_important_texts, name)

In [394]:
# PLease choose from these 3 companies, we have the data ready for only these 3 companies
''' ASL, FICO, WDC '''
full_pipeline('FICO')

 The highlights of the discussion we are going to see for the company 'FICO' 
 
 We are currently looking in the year : 2022 
 
 Currently seeing important points for the quarter : 1  
 point 1 :   But that was more than offset by growth in other areas 
 point 2 :   As always, we continue to be focused on shareholder value 
 point 3 :   But if you think about the total revenue increase that was not volume 
 point 4 :   where revenues growing rapidly 
 point 5 :   Total revenues for the first quarter was $322 million, an increase of 3% over the prior year or 9% after adjusting for the divestiture of our collections and recovery product line last year 
 
 Currently seeing important points for the quarter : 2  
 point 1 :   Total revenue for the second quarter was $357 million, an increase of 8% over the prior year or 13% after adjusting for the divestiture of our Collections and Recovery product line last June 
 point 2 :   And then perhaps -- obviously, there's a lot of uncertainties in